# Case Study

## Initial data analysis and preprocessing

In [305]:
# just tweets mentioning Dell during first three quarters of 2022 
# https://www.kaggle.com/datasets/ankitkumar2635/dell-tweets-2022
import pandas as pd

df_dell = pd.read_csv("./analysis_data/First three qtr Dell tweets.csv")
df_dell.dropna(inplace=True)
df_dell.drop_duplicates(inplace=True)
df_dell.head()

In [306]:
len(df_dell)

In [307]:
import pandas as pd
import re
from collections import Counter

# counting most frequent mentions
df_dell["mentions"] = df_dell["Text"].apply(lambda x: re.findall(r'@\S+', str(x)) if pd.notna(x) else [])
all_mentions = [mention for mentions in df_dell["mentions"] for mention in mentions]
mention_counts = pd.Series(Counter(all_mentions))
mention_counts = mention_counts.sort_values(ascending=False)
for index, value in mention_counts[:100].items():
    print(f"{index}: {value}")

In [308]:
# replacing urls and user mentions while keeping select mentions
import re
from urlextract import URLExtract

mentions_to_keep = ["Dell", "Delltech", "Dellcares", "Dell,", "DellOutlet", "Dell_IN", "DellCaresPRO", "DellTechIndia",
                    "Dell's", "DellServices", "dellindia", "DellUK", "DellTechZA", "FedEx", "HPIndia", "PwC", "Delta",
                    "Deloitte", "AlienwareTech", "AMD", "Acer", "Tesla", "comcast", "Meta", "nvidia", "Ford", "YouTube",
                    "HP", "Microsoft", "Apple", "Logitech", "ubuntu", "playstation", "Windows", "Oracle", "intelcanada",
                    "Google", "ASUS", "Lenovo", "Tesla", "Intel", "Alienware", "Emc", "WWEUniverse", "AmericanAir",
                    "Zipchair", "ZipchairGaming", "RapSnacksNow", "ATT", "IBM", "amazon", "Walmart", "Cisco", 
                    "MichaelDell", "AlokOhrie", "Elonmusk", "JohnCena", "TheRock", "steveaustinBSR", "_KennyRogers",
                    "VMware", "SouthwestAir", "Lenovo_in"]

extractor = URLExtract()

def format_tweet_new_keep(tweet):

    tokens = tweet.split()
    tokens = [token[1:].capitalize() if token.startswith("@") and token[1:].capitalize() in mentions_to_keep else token for token in tokens]

    tweet = " ".join(tokens)

    urls = extractor.find_urls(tweet)
    for url in urls:
        tweet = tweet.replace(url, "{{URL}}")

    tweet = re.sub(r'@\S+', "{{MENTION}}", tweet)

    return tweet

df_dell["text"] = df_dell.apply(lambda x: format_tweet_new_keep(str(x["Text"])), axis=1)
df_dell.head()

In [309]:
df_dell["tokens"] = df_dell["text"].apply(lambda x: x.split())
df_dell["tokens"].str.len().agg(["mean","max","std"])

In [310]:
df_dell = df_dell[["Datetime", "text", "tokens"]]
df_dell.head()

In [311]:
len(df_dell)

In [312]:
df_dell.to_json("./analysis_data/dell_cs_processed.json", orient="records")

Predicting entity- and sentiment-labels is performed in the respective notebooks

## processing the predictions

In [392]:
df_dell_p = pd.read_json("./analysis_data/dell_cs_after_predictions.json", orient="records")
df_dell_p["sentiment_labels"] = df_dell_p["bert_pred_sa"].replace({0 : "negative", 1 : "neutral", 2 : "positive"})
df_dell_p["Datetime"] = pd.to_datetime(df_dell_p["Datetime"])
df_dell_p.head()

In [393]:
sum(df_dell_p["tokens"].str.len() != df_dell_p["covid_bert_pred"].str.len())

In [394]:
# removing the rows with mismatched lengths (caused by truncation to max length of 128 tokens)
df_dell_p = df_dell_p[df_dell_p["tokens"].str.len() == df_dell_p["covid_bert_pred"].str.len()]
print(f"Number of rows predicted in their entirety: {len(df_dell_p)}")

In [395]:
df_dell_p[df_dell_p["bert_pred_sa"].isin([0, 1, 2])].shape[0]

In [396]:
print(df_dell_p["sentiment_labels"].value_counts())

In [397]:
# converting the tags of recognized instances of Dell mentions from tag-column to 10 ("O"-tag)
own_company_names = ["Dell", "Delltech", "Dellcares", "Dell,", "DellOutlet", "Dell_IN", "DellCaresPRO", "DellTechIndia",
                    "Dell's", "DellServices", "dellindia", "DellUK", "DellTechZA"]

for index, row in df_dell_p.iterrows():
    tokens = row["tokens"]
    labels = row["covid_bert_pred"]
    for i in range(len(tokens)):
        if tokens[i] in own_company_names and labels[i] != 10:
            labels[i] = 10
            modified = True
    
    if modified:
        df_dell_p.at[index, "covid_bert_pred"] = labels

In [398]:
entity_dict = {
    0: "B-corporation",
    1: "B-event",
    2: "B-location",
    3: "B-person",
    4: "B-product",
    5: "I-corporation",
    6: "I-event",
    7: "I-location",
    8: "I-person",
    9: "I-product",
    10: "O"
}

label_list = list(entity_dict.values())
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

df_dell_p["ner_labels"] = df_dell_p["covid_bert_pred"].apply(lambda x: [id_to_label[i] for i in x])

In [399]:
def get_types_from_list(lst):
    r = []
    for entry in lst:
        if entry != "O":
            parts = entry.split("-", 1)
            if len(parts) > 1:
                entity_type = parts[1]
                if entity_type not in r:
                    r.append(entity_type)
    return r

df_dell_p["entity_types"] = df_dell_p["ner_labels"].apply(get_types_from_list)
df_dell_p.head()

In [400]:
df_dell_p["entity_types"].explode().value_counts()

## visual explorations

In [401]:
df_dell_p.head()

### bar chart

In [402]:
import matplotlib.pyplot as plt

unique_entities = ["corporation", "product", "person", "location", "event"]

def mentioned_entities_with_sentiment_breakdown(df_results):
    dic = {}
    for entity_type in unique_entities:
        df_results["entity_mentioned"] = df_results["entity_types"].apply(lambda x: entity_type in x)
        filtered_df = df_results[df_results["entity_mentioned"]==True]
        percentage = len(filtered_df)/len(df_results)
        dic[entity_type] = percentage

    print(dic)

    entity_dic = {}
    for entity_type in unique_entities:
        df_results["entity_mentioned"] = df_results["entity_types"].apply(lambda x: entity_type in x)
        filtered_df = df_results[df_results["entity_mentioned"]==True]
        percentage = len(filtered_df)/len(df_results)
        entity_dic[entity_type] = percentage

    sentiment_dic = {}
    for entity_type in unique_entities:
        df_results["entity_mentioned"] = df_results["entity_types"].apply(lambda x: entity_type in x)
        filtered_df = df_results[df_results["entity_mentioned"]==True]
        sentiment_dic[entity_type] = {"negative": 0, "neutral": 0, "positive": 0}

        total_count = len(filtered_df)
        negative_count = len(filtered_df[filtered_df["sentiment_labels"] == "negative"])
        neutral_count = len(filtered_df[filtered_df["sentiment_labels"] == "neutral"])
        positive_count = len(filtered_df[filtered_df["sentiment_labels"] == "positive"])
        
        sentiment_dic[entity_type] = {
            "negative": (negative_count / total_count) * dic[entity_type],
            "neutral": (neutral_count / total_count) * dic[entity_type],
            "positive": (positive_count / total_count) * dic[entity_type]
        }
        
    entity_types = list(entity_dic.keys())
    x = range(len(entity_types))
    total_percentages = list(entity_dic.values())

    negative_values = [sentiment_dic[entity]["negative"] for entity in entity_types]
    neutral_values = [sentiment_dic[entity]["neutral"] for entity in entity_types]
    positive_values = [sentiment_dic[entity]["positive"] for entity in entity_types]

    fig, ax = plt.subplots(figsize=(12, 8))

    p1 = ax.bar(x, negative_values, align="center", label="Negative", color="blue")
    p2 = ax.bar(x, neutral_values, align="center", bottom=negative_values, label="Neutral", color="orange")
    p3 = ax.bar(x, positive_values, align="center", 
                bottom=[p+n for p, n in zip(negative_values, neutral_values)], 
                label="Positive", color="green")

    for i, percentage in enumerate(total_percentages):
        v = f"{round(percentage*100, 2)}%"
        ax.text(i, percentage, v, ha="center")

    ax.set_xticks(x)
    ax.set_xticklabels(entity_types)
    ax.set_title("Percentage of postings mentioning the entity type with sentiment breakdown")
    ax.legend()
    ax.grid(axis="y", linestyle="--")

    plt.tight_layout()
    plt.show()

mentioned_entities_with_sentiment_breakdown(df_dell_p)

conclusions: 
- corporations and products are mentioned the most, in over 35% of all posts
- of corporation- and product-posts, negativity is the most common sentiment
- the large majority of posts contain sentiment, neutrality is quite rare

### word clouds

In [403]:
# creating wordclouds to investigate the vocabulary of negative corporation- and product posts
from wordcloud import WordCloud, STOPWORDS

stopwords = set(STOPWORDS)

#adding frequently mentioned uninteresting words to the stopwords to not be included in the wordcloud
stopwords.update(["mention", "rt", "link", "url", "quot", "dell", "dellcares", "dellcare", "delltech"])

def create_wordcloud(df, entity, sentiment, text_col="text"):
    words = ""

    df["entity_mentioned"] = df["entity_types"].apply(lambda x: entity in x)
    filtered_df = df[(df["sentiment_labels"]==sentiment) & (df["entity_mentioned"]==True)]

    for val in filtered_df[text_col]:
        val = str(val)
        tokens = val.split()
        
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
        
        words += " ".join(tokens)+" "
    
    wordcloud = WordCloud(width=800, 
                          height=800,
                        background_color="white",
                        stopwords=stopwords,
                        min_font_size=10).generate(words)
                    
    plt.figure(figsize = (5, 5), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.title(f"Wordcloud of all {sentiment} postings mentioning a {entity}")
    plt.show()

In [404]:
create_wordcloud(df_dell_p, "corporation", "negative")

In [405]:
create_wordcloud(df_dell_p, "corporation", "positive")

In [406]:
create_wordcloud(df_dell_p, "product", "negative")

In [407]:
create_wordcloud(df_dell_p, "product", "positive")

conclusions:
- corporation: general problems related to technology, sometimes related to competitors
- product: warranty often related, apparently mostly general issues with products

### entity type pair sentiment heatmap

In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

all_entity_types = sorted(set([entity for sublist in df_dell_p['entity_types'] for entity in sublist]))

sentiment_matrix = pd.DataFrame(
    index=all_entity_types,
    columns=all_entity_types,
    dtype=float
)

for i, type1 in enumerate(all_entity_types):
    for j, type2 in enumerate(all_entity_types):
        if i <= j:
            mask = df_dell_p['entity_types'].apply(lambda x: type1 in x and type2 in x)
            if mask.sum() > 0:
                avg_sentiment = df_dell_p.loc[mask, 'bert_pred_sa'].mean()
                sentiment_matrix.loc[type1, type2] = avg_sentiment
                if i != j:  
                    sentiment_matrix.loc[type2, type1] = avg_sentiment

# adjusting the order of the y-axis
sentiment_matrix_aligned = sentiment_matrix.iloc[::-1]

plt.figure(figsize=(8, 8))
sns.heatmap(
    sentiment_matrix_aligned,  
    annot=True,
    fmt=".2f",
    cmap=plt.cm.RdYlGn,
    center=1,
    cbar_kws={'label': 'Average Sentiment Score'}
)
plt.title("Entity Type Pair Sentiment Heatmap")
plt.xlabel("Entity Type")
plt.ylabel("Entity Type")

plt.tight_layout()
plt.show()

conclusions:
- events mentioned in positive discussions regardless of other entities present
- location-person the next strongest pairing, persons in general also quite positive
- product-corporation is the most negatively associated pairing

### top 10 instances with sentiment

In [409]:
# getting instance based insight to investigate most frequently mentioned instances of entities
import regex as re

instances_to_remove = own_company_names + ["Mention", "Url", "Computer", "Laptop", "Laptops", "Pc", "Monitor", "2022"]

entity_mapping = {
    "person": {
        "Elon": "Elon Musk",
        "Elonmusk": "Elon Musk",
        "Elon musk": "Elon Musk"
    },
    "location": {
        "Tx" : "Texas",
        "Us" : "Usa"
    },
    "event": {
        "Ces" : "Ces 2022",
        "New year" : "New years" 
    },
    "corporation": {
         "HPIndia" : "HP",
         "Lenovo_in" : "Lenovo",
         "intelcanada" : "Intel"
    }
#         "Dell computers": "Dell",
#         "Dell tech": "Dell Technologies",
#         "Dell technology": "Dell Technologies",
#     },
#     "product": {
#         "Xps13": "XPS 13",
#         "Xps 13plus": "XPS 13 Plus",
#         "Alienware": "Alienware laptop",
#     }
}

def get_instance_dic(df):
    unique_entities = ["corporation", "product", "person", "location", "event"]
    instance_dic = {}
    
    for type in unique_entities:
        type_list = []
        
        for index, row in df.iterrows():
            tokens = row["tokens"]
            labels = row["ner_labels"]
            
            i = 0
            while i < len(labels):
                if labels[i] == f"B-{type}":
                    entity_tokens = [tokens[i]]
                    j = i + 1
                    
                    while j < len(labels) and labels[j] == f"I-{type}":
                        entity_tokens.append(tokens[j])
                        j += 1
                    
                    complete_entity = " ".join(entity_tokens)
                    complete_entity = re.sub(r'[^a-zA-Z0-9\s]', '', complete_entity).capitalize().strip().removesuffix("'s")
                    
                    # applying manual mapping to group select instances together
                    if type in entity_mapping and complete_entity in entity_mapping[type]:
                        complete_entity = entity_mapping[type][complete_entity]
                    
                    if complete_entity not in instances_to_remove:
                        type_list.append(complete_entity)
                    
                    i = j
                else:
                    i += 1
        
        instance_dic[type] = type_list
    
     # remove trailing "s" if instance is already known with 2nd pass through instance_dic to replace "Googles" with "Google"
    for type in instance_dic:
        unique_instances = set(instance_dic[type])  
        new_type_list = []

        for entity in instance_dic[type]:
            if entity.endswith("s"):
                singular_candidate = entity[:-1]  
                if singular_candidate in unique_instances:
                    entity = singular_candidate 
            
            new_type_list.append(entity)

        instance_dic[type] = new_type_list

    return instance_dic

instance_dic = get_instance_dic(df_dell_p)
print(instance_dic["product"])
    
for key in instance_dic.keys():
    print(f"Detected instances of {key}: {len(instance_dic[key])}")

In [410]:
def get_top_instances(instance_dic, top_n=10):
    top_instances = {}
    
    for entity_type, instances in instance_dic.items():
        instance_counts = {}
        for instance in instances:
            instance_counts[instance] = instance_counts.get(instance, 0) + 1
        
        sorted_instances = sorted(instance_counts.items(), key=lambda x: x[1], reverse=True)
        
        top_n_instances = sorted_instances[:top_n]
        
        top_instances[entity_type] = top_n_instances

    return top_instances

top_10_instances = get_top_instances(instance_dic, top_n=10)
for key in top_10_instances.keys():
    print(f"{key}: {top_10_instances[key]}")

In [ ]:
# adding sentiment to the entity-instances, not including instances that could not be found in the text-column
def add_sentiment_to_instances(entity_dict, df):
    result = {}
    
    for entity_type, instances in entity_dict.items():
        instances_with_sentiment = []
        instances_not_found = []
        
        for entity, count in instances:
            entity_lower = entity.lower()
            
            # filtering in text-column to detect multi-token instances
            mask = df["text"].apply(lambda x: entity_lower in x.lower())
            
            if mask.sum() > 0:
                avg_sentiment = df.loc[mask, "bert_pred_sa"].mean()
                instances_with_sentiment.append((entity, count, avg_sentiment))
            else:
                instances_not_found.append(entity)
        
        if instances_not_found:
            print(f"{len(instances_not_found)} {entity_type} entities not found in text: {instances_not_found}")
            
        result[entity_type] = instances_with_sentiment
    
    return result

top_10_instances2 = add_sentiment_to_instances(top_10_instances, df_dell_p)
for key in top_10_instances2.keys():
    print(f"{key}: {top_10_instances2[key]}")

In [412]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

def barchart_top10_instances_with_sentiment(entity):
    instances = [item[0] for item in top_10_instances2[entity]]
    counts = [item[1] for item in top_10_instances2[entity]]
    sentiments = [item[2] for item in top_10_instances2[entity]]

    cmap = plt.cm.RdYlGn  
    norm = mcolors.Normalize(vmin=0, vmax=2) # color-range from 0 to 2

    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(range(len(instances)), counts, align="center")

    for i, bar in enumerate(bars):
        if np.isnan(sentiments[i]):
            bar.set_color("black")  
        else:
            bar_color = cmap(norm(sentiments[i]))
            bar.set_color(bar_color)

    for i, v in enumerate(counts):
        ax.text(i, v + 1, str(v), ha='center', va='bottom')

    for i, sentiment in enumerate(sentiments):
        ax.text(i, -5, f"({sentiment:.2f})", ha='center', va='top', color='blue')

    ax.set_xticks(range(len(instances)))
    ax.set_xticklabels(instances, rotation=45, ha='right')
    ax.set_xlabel(f"{entity.capitalize()}s")
    ax.set_ylabel(f"Posts mentioning this {entity}")
    ax.set_title(f"Top 10 most frequently mentioned {entity}s (colored by sentiment)")

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label('Sentiment Score')

    plt.tight_layout()
    plt.show()

barchart_top10_instances_with_sentiment("product")

In [413]:
barchart_top10_instances_with_sentiment("corporation")

In [414]:
barchart_top10_instances_with_sentiment("person")

In [415]:
barchart_top10_instances_with_sentiment("location")

In [416]:
barchart_top10_instances_with_sentiment("event")

conclusions:
- corporations: posts mentioning "intel" are the most positive out of the top 10 instances, with Google and Tesla being mentioned in more negative posts
- products: posts mentioning Dell laptops are mostly negative, Linux is the most positively discussed product of the top 10 most mentioned (average of 1.24)
- persons: Elon Musk is by far the most discussed person in either a neutral or highly diverse fashion, Michael likely refers to Dell-CEO Michael Dell
- location: India most frequently mentioned and associated with negative sentiment, Austin is the most positively discussed location in top 10
- event: most mentioned event is the CDW executive summit, holidays are also in top 10

### time-based plots

In [417]:
# getting general time based overview
import seaborn as sns

plt.figure(figsize=(12, 6))

# plotting the overall sentiment using the weekly average
weekly_sentiment = df_dell_p.groupby(pd.Grouper(key="Datetime", freq="W"))["bert_pred_sa"].mean().reset_index()
plt.plot(weekly_sentiment["Datetime"], weekly_sentiment["bert_pred_sa"], marker="o", linestyle="dashed", color="black", label="overall")

# plotting the average sentiment for all rows mentioning the entities, adding colors dynamically
colors = sns.color_palette("Set2", n_colors=len(unique_entities))
for i, entity_type in enumerate(unique_entities):
    entity_df = df_dell_p[df_dell_p["entity_types"].apply(lambda x: entity_type in x)]
    weekly_sentiment_filtered = entity_df.groupby(pd.Grouper(key="Datetime", freq="W"))["bert_pred_sa"].mean().reset_index()
    plt.plot(weekly_sentiment_filtered["Datetime"], weekly_sentiment_filtered["bert_pred_sa"], 
             marker="o", linestyle="-", color=colors[i], label=entity_type)

plt.legend()
plt.title("Sentiment Over Time by Entity Type")
plt.xlabel("Date")
plt.ylabel("Average Sentiment")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [418]:
# more specific time-based insights
def plot_sentiment_for_entities_over_time(df, entities):
    plt.figure(figsize=(12, 6))

    if isinstance(entities, str):
        entity_df = df[df["entity_types"].apply(lambda x: entities in x)]

        weekly_sentiment = entity_df.groupby(pd.Grouper(key="Datetime", freq="W"))["bert_pred_sa"].mean().reset_index()
        plt.plot(weekly_sentiment["Datetime"], weekly_sentiment["bert_pred_sa"], 
                 marker="o", linestyle="-", color="blue", label=entities)

        plt.legend()
        plt.title(f"Sentiment Over Time for Posts mentioning a {entities.capitalize()}")
    
    if isinstance(entities, list):
        entity_df = df[df["entity_types"].apply(lambda x: all(ent in x for ent in entities))]

        weekly_sentiment = entity_df.groupby(pd.Grouper(key="Datetime", freq="W"))["bert_pred_sa"].mean().reset_index()
        plt.plot(weekly_sentiment["Datetime"], weekly_sentiment["bert_pred_sa"], 
                 marker="o", linestyle="-", color="blue", label=", ".join([ent for ent in entities]))

        plt.legend()
        plt.title(f"Sentiment Over Time for Posts mentioning {", ".join([ent.capitalize() for ent in entities])}")
        
    plt.xlabel("Date")
    plt.ylabel("Average Sentiment")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_sentiment_for_entities_over_time(df_dell_p, "person")

In [419]:
plot_sentiment_for_entities_over_time(df_dell_p, ["corporation", "product"])

conclusions:
- overall sentiment is relatively consistently hovering around 0.9 with the peak being reached in May 2022 (~1.1), lowest overall sentiment in January 2022 (~0.75)
- posts mentioning events are generally the most positive except for a negative start at the beginning of 2022, but relatively small sample size (704 posts detected as mentioning an event)
- persons are discussed with mostly positive sentiment over entire time period (majority of weekly averages > 1)